In [3]:
import numpy as np
import pandas as pd
from itertools import product, chain
import re

In [33]:
# with open('Day8_input_test.txt') as f:
with open('Day8_input.txt') as f:
    raw = f.readlines()
raw[-5:]


['acc +12\n', 'nop -245\n', 'acc +26\n', 'acc +15\n', 'jmp +1']

In [34]:
code = [r.strip().split(' ') for r in raw]
code = [[line[0], int(line[1])] for line in code]
code[-5:]

[['acc', 12], ['nop', -245], ['acc', 26], ['acc', 15], ['jmp', 1]]

In [37]:
def gen_codedf():
    codedf = pd.DataFrame(np.array(code), columns=['op', 'arg'])
    codedf['acc'] = None
    codedf['arg'] = codedf['arg'].astype(int)
    return codedf
gen_codedf()

,op,arg,acc
0,acc,3,None
1,jmp,599,None
2,nop,311,None
3,jmp,605,None
4,acc,-3,None
...,...,...,...
636,acc,12,None
637,nop,-245,None
638,acc,26,None
639,acc,15,None


# Part 1

In [36]:
def do_op(df, index, current_acc):
    op, arg, acc = df.iloc[index]    
    next_index, new_acc = None, current_acc
    if op == 'jmp':
        next_index = index+arg
    elif op == 'acc':
        new_acc = current_acc + arg
        next_index = index+1
    elif op == 'nop':
        next_index = index+1
    else:
        raise ValueError
    df.loc[index, 'acc'] = new_acc
    return next_index
    

In [38]:
codedf = gen_codedf()
indexs, prev_acc = [0], 0
while True:
    next_acc = codedf.loc[indexs[-1], 'acc']
    if next_acc is not None:
        print(f'Found Acc = {next_acc} at Index: {indexs[-1]}. Previous index, acc = {indexs[-2], codedf.loc[indexs[-2], "acc"]}')
        break
    if len(indexs) > 1:
        acc = codedf.loc[indexs[-2], 'acc']
    else:
        print(f'Starting with acc = 0')
        acc = 0
    indexs.append(do_op(codedf, indexs[-1], acc))
#     print(codedf, '\n\n')
    
    
    

Starting with acc = 0
Found Acc = 58 at Index: 108. Previous index, acc = (463, 1586)


# Part 2

In [68]:
codedf = gen_codedf()

line_locs = np.where(np.logical_or(codedf['op'] == 'jmp', codedf['op'] == 'nop'))[0]

indexs, prev_acc = [0], 0
line_changed = codedf.last_valid_index()
while True:
    if indexs[-1] > codedf.last_valid_index():
        print(f'Code finished with acc = {codedf.loc[indexs[-2], "acc"]}')
        break
    next_acc = codedf.loc[indexs[-1], 'acc']
    if next_acc is not None:
#         print(f'Found index, acc = {indexs[-1], next_acc}. Previous index, acc = {indexs[-2], codedf.loc[indexs[-2], "acc"]}\nGoing to change a different line')
        line_changed = line_locs[np.where(line_locs < line_changed)][-1]
        codedf = gen_codedf()
        indexs, prev_acc = [0], 0
        current_op = codedf.loc[line_changed, 'op']
        if current_op == 'jmp':
            codedf.loc[line_changed, 'op'] = 'nop'
        elif current_op == 'nop':
            codedf.loc[line_changed, 'op'] = 'jmp'
        else:
            raise ValueError       
        print(f'Running again with line {line_changed} changed')
    if len(indexs) > 1:
        acc = codedf.loc[indexs[-2], 'acc']
    else:
#         print(f'Starting with acc = 0')
        acc = 0
    indexs.append(do_op(codedf, indexs[-1], acc))

Running again with line 637 changed
Running again with line 635 changed
Running again with line 634 changed
Running again with line 633 changed
Running again with line 630 changed
Running again with line 626 changed
Running again with line 625 changed
Running again with line 621 changed
Running again with line 620 changed
Running again with line 617 changed
Running again with line 616 changed
Running again with line 612 changed
Running again with line 611 changed
Running again with line 607 changed
Running again with line 602 changed
Running again with line 601 changed
Running again with line 600 changed
Running again with line 599 changed
Running again with line 597 changed
Running again with line 595 changed
Running again with line 594 changed
Running again with line 593 changed
Running again with line 591 changed
Running again with line 590 changed
Running again with line 587 changed
Running again with line 585 changed
Running again with line 584 changed
Running again with line 581 

In [60]:
line_locs = np.where(np.logical_or(codedf['op'] == 'jmp', codedf['op'] == 'nop'))[0]
print(line_locs)
line_locs[np.where(line_locs < 637)][-1]

[  1   2   3   7   8  10  13  14  15  18  19  22  23  27  29  31  33  37
  38  41  42  43  44  45  48  49  51  52  53  54  55  56  57  59  61  65
  66  70  71  74  76  78  79  80  83  84  85  89  91  92  95  97  98 100
 101 105 106 107 110 111 112 116 117 118 122 123 126 129 130 134 136 138
 140 143 146 147 149 151 152 153 154 155 156 159 160 163 165 166 169 174
 176 177 179 181 184 185 187 188 191 195 196 198 201 202 204 206 209 213
 215 218 220 221 223 224 226 227 230 235 237 239 241 242 244 247 249 250
 251 255 256 259 263 267 270 271 273 275 276 278 279 281 283 284 288 289
 290 291 295 296 297 299 300 301 303 305 308 311 312 313 314 319 320 322
 327 330 335 336 337 339 340 343 346 348 350 352 356 358 360 362 364 369
 372 374 376 380 381 382 386 389 391 393 394 397 398 401 405 406 409 410
 411 416 420 421 422 423 424 425 426 428 429 430 435 436 438 440 441 445
 446 447 448 449 450 452 454 459 463 467 469 470 471 473 474 475 476 478
 479 481 484 485 486 489 493 494 495 496 497 499 50

635